In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.loader import DataLoader


from data_loaders import preproccess_data, generate_scaffold_split, df_to_graph_list, get_scaffolds
from graphnn1 import GCN1

from sklearn.metrics import r2_score
import numpy as np
import scipy.stats as stats

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
file_path = 'data/curated-solubility-dataset.csv'
df = preproccess_data(file_path)


df['scaffold'] = df['mol'].apply(get_scaffolds)

# scaffolds to get train, val, text
train_idx, val_idx, test_idx = generate_scaffold_split(df)

# Split the dataframe into train, val, and test
train_df = df.iloc[train_idx]
val_df = df.iloc[val_idx]
test_df = df.iloc[test_idx]

# df to graph list
train_graph_list = df_to_graph_list(train_df)
val_graph_list = df_to_graph_list(val_df)
test_graph_list = df_to_graph_list(test_df)

[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not removing hydrogen atom without neighbors
[12:36:43] WARNING: not r

In [11]:
train_loader = DataLoader(train_graph_list, batch_size=32, shuffle=True)
val_loader = DataLoader(val_graph_list, batch_size=32, shuffle=False)
test_loader = DataLoader(test_graph_list, batch_size=32, shuffle=False)

In [ ]:
# Set seed for reproducibility
torch.manual_seed(42)

num_node = train_graph_list[0].x.shape[1]
edge_attr = train_graph_list[0].edge_attr.shape[1]
u_d = train_graph_list[0].u.shape[1]

model = GCN1(num_node_features=num_node,
            edge_attr_dim=edge_attr,
            u_dim=u_d, 
            hidden_dim=64, 
            output_dim=1).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 100  
for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        target = data.y.view(data.num_graphs, -1).to(device)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.num_graphs
    train_loss /= len(train_loader.dataset)
    
    # Validation step
    model.eval()
    all_preds, all_targets = [], []
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.to(device)
            output = model(data)
            target = data.y.view(data.num_graphs, -1).to(device)
            loss = criterion(output, target) #get loss based on criterion
            val_loss += loss.item() * data.num_graphs
            all_preds.extend(output.cpu().numpy())
            all_targets.extend(target.cpu().numpy())
    val_loss /= len(val_loader.dataset) #compute validation loss
    val_rmse = val_loss ** 0.5
    
    # Compute R^2
    all_preds = np.array(all_preds).flatten()
    all_targets = np.array(all_targets).flatten()
    r2 = r2_score(all_targets, all_preds)

    # Compute 95% Confidence Interval for RMSE
    confidence = 0.95
    squared_errors = (all_preds - all_targets) ** 2
    mean_se = np.mean(squared_errors)
    se = stats.sem(squared_errors)
    interval = stats.t.interval(confidence, len(squared_errors)-1, loc=mean_se, scale=se)
    ci_lower, ci_upper = np.sqrt(interval[0]), np.sqrt(interval[1])
    
    print(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}, Val RMSE: {val_rmse:.4f}, R²: {r2:.4f}, CI (95%):[{ci_lower:.4f}, {ci_upper:.4f}]")

Epoch: 1, Train Loss: 331.0987, Val RMSE: 4.2636, R^2: -20.2148, CI (95%):[3.9652, 4.5425]
Epoch: 2, Train Loss: 36.3586, Val RMSE: 3.2362, R^2: -11.2219, CI (95%):[2.9107, 3.5318]
Epoch: 3, Train Loss: 20.6112, Val RMSE: 3.0511, R^2: -9.8640, CI (95%):[2.8019, 3.2814]
Epoch: 4, Train Loss: 12.4701, Val RMSE: 2.2234, R^2: -4.7691, CI (95%):[2.0613, 2.3744]
Epoch: 5, Train Loss: 7.1693, Val RMSE: 1.9541, R^2: -3.4562, CI (95%):[1.7825, 2.1118]
Epoch: 6, Train Loss: 4.5955, Val RMSE: 1.5152, R^2: -1.6794, CI (95%):[1.4201, 1.6047]
Epoch: 7, Train Loss: 2.4687, Val RMSE: 1.0116, R^2: -0.1941, CI (95%):[0.9346, 1.0830]
Epoch: 8, Train Loss: 1.4673, Val RMSE: 0.8270, R^2: 0.2018, CI (95%):[0.7658, 0.8840]
Epoch: 9, Train Loss: 1.0398, Val RMSE: 0.7148, R^2: 0.4038, CI (95%):[0.6668, 0.7598]
Epoch: 10, Train Loss: 0.7966, Val RMSE: 0.6817, R^2: 0.4577, CI (95%):[0.6397, 0.7213]
Epoch: 11, Train Loss: 0.6935, Val RMSE: 0.6281, R^2: 0.5396, CI (95%):[0.5964, 0.6582]
Epoch: 12, Train Loss: 0.62

In [15]:
# Testing
model.eval()
test_loss = 0
all_preds, all_targets = [], []

with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        output = model(data)
        target = data.y.view(data.num_graphs, -1).to(device)
        loss = criterion(output, target)
        test_loss += loss.item() * data.num_graphs
        all_preds.extend(output.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
test_loss /= len(test_loader.dataset)
test_rmse = test_loss ** 0.5

# Compute R^2
all_preds = np.array(all_preds).flatten()
all_targets = np.array(all_targets).flatten()
r2 = r2_score(all_targets, all_preds)

# Compute 95% Confidence Interval for RMSE
confidence = 0.95
squared_errors = (all_preds - all_targets) ** 2
mean_se = np.mean(squared_errors)
se = stats.sem(squared_errors)
interval = stats.t.interval(confidence, len(squared_errors)-1, loc=mean_se, scale=se)
ci_lower, ci_upper = np.sqrt(interval[0]), np.sqrt(interval[1])

print(f"Test RMSE: {test_rmse:.4f}, R²: {r2:.4f}, CI (95%): [{ci_lower:.4f}, {ci_upper:.4f}]")


Test RMSE: 0.7463, R²: 0.1641, CI (95%): [0.7122, 0.7789]
